# Wstęp do analizy danych i uczenia maszynowego
## 3. Regresja liniowa, pipeliny w scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

sns.set(style="whitegrid")

import warnings
warnings.filterwarnings("ignore")

W zbiorze danych `mpg` (miles per gallon) z biblioteki seaborn znajdują się informacje o samochodach, w tym ich zużycie paliwa (mpg), które chcemy przewidzieć na podstawie innych cech pojazdów.

Na początku przyjrzmy się temu zbiorowi danych.

In [ ]:
df = sns.load_dataset("mpg")

df.head()

Niektóre ze zmiennych są kategoryczne, przed stworzeniem modelu będziemy musieli je zakodować (np. za pomocą one-hot encoding).

In [ ]:
df.info()

In [ ]:
df.isna().sum()

W jednej z kolumn pojawiają się wartości brakujące, które również musimy obsłużyć przed trenowaniem modelu.

Przed przetwarzaniem danych podzielmy je na zbiór treningowy i testowy.

In [ ]:
X = df.drop("mpg", axis=1)
y = df["mpg"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
X_train.head(), X_test.head(), y_train.head(), y_test.head()

W sprawnym przetwarzaniu danych i trenowaniu modeli w scikit-learn bardzo pomocne są pipeliny. Umożliwiają one łączenie wielu kroków przetwarzania danych i trenowania modelu w jeden obiekt, co znacznie upraszcza kod.

Na początku zdefiniujemy, które ze zmiennych są kategoryczne, a które numeryczne. Następnie stworzymy osobne przetwarzania dla każdej z tych grup zmiennych, a na końcu połączymy je w jeden pipeline.

In [ ]:
num_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

print("Wszystkie cechy:", X_train.columns.tolist())
print("Numeryczne cechy:", num_features)
print("Kategoryczne cechy:", cat_features)

W ramach przetwarzania danych dla zmiennych numerycznych uzupełnimy brakujące wartości medianą i standaryzujemy cechy za pomocą `StandardScaler`. Dla zmiennych kategorycznych uzupełnimy brakujące wartości najczęściej występującą kategorią i zakodujemy je za pomocą `OneHotEncoder`.

Po więcej szczegółów odnośnie modułu `Pipeline` i `ColumnTransformer` odsyłam do dokumentacji scikit-learn:
- [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
- [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

In [ ]:
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])

data_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor)
])

# Dopasowanie pipeline do danych treningowych
data_pipeline.fit(X_train)

# Użycie pipeline do utworzenia przetworzonych ramek danych
feature_names = (
    pd.Index(data_pipeline.named_steps['preprocessor'].get_feature_names_out())
    .str.replace("num__", "", regex=False)
    .str.replace("cat__", "", regex=False)
)

X_train_processed = pd.DataFrame(data_pipeline.transform(X_train).toarray(),
                                columns=feature_names)

X_test_processed = pd.DataFrame(data_pipeline.transform(X_test).toarray(),
                                columns=feature_names)

X_train_processed.head()

Otrzymaliśmy w ten sposób nowy, gotowy do trenowania modelu zbiór cech. Teraz możemy stworzyć i wytrenować model regresji liniowej.

In [ ]:
model = LinearRegression(positive=False)

model.fit(X_train_processed, y_train)
y_pred = model.predict(X_test_processed)
mse_train = mean_squared_error(y_train, model.predict(X_train_processed))
r2_train = r2_score(y_train, model.predict(X_train_processed))
mse_test = mean_squared_error(y_test, y_pred)
r2_test = r2_score(y_test, y_pred)

print(f"Linear Regression - Mean Squared Error, Train: {mse_train:.2f}, Test: {mse_test:.2f}")
print(f"Linear Regression - R^2 Score, Train: {r2_train:.2f}, Test: {r2_test:.2f}")

In [ ]:
model_lasso = Lasso(alpha=1)

model_lasso.fit(X_train_processed, y_train)
y_pred_lasso = model_lasso.predict(X_test_processed)
mse_lasso_train = mean_squared_error(y_train, model_lasso.predict(X_train_processed))
r2_lasso_train = r2_score(y_train, model_lasso.predict(X_train_processed))
mse_lasso_test = mean_squared_error(y_test, y_pred_lasso)
r2_lasso_test = r2_score(y_test, y_pred_lasso)

print(f"Lasso Regression - Mean Squared Error, Train: {mse_lasso_train:.2f}, Test: {mse_lasso_test:.2f}")
print(f"Lasso Regression - R^2 Score, Train: {r2_lasso_train:.2f}, Test: {r2_lasso_test:.2f}")

In [ ]:
model_ridge = Ridge(alpha=1)

model_ridge.fit(X_train_processed, y_train)
y_pred_ridge = model_ridge.predict(X_test_processed)
mse_ridge_train = mean_squared_error(y_train, model_ridge.predict(X_train_processed))
r2_ridge_train = r2_score(y_train, model_ridge.predict(X_train_processed))
mse_ridge_test = mean_squared_error(y_test, y_pred_ridge)
r2_ridge_test = r2_score(y_test, y_pred_ridge)

print(f"Ridge Regression - Mean Squared Error, Train: {mse_ridge_train:.2f}, Test: {mse_ridge_test:.2f}")
print(f"Ridge Regression - R^2 Score, Train: {r2_ridge_train:.2f}, Test: {r2_ridge_test:.2f}")

W zwykłej regresji na zbiorze testowym dzieje się coś dziwnego, zignorujmy to na razie, nie rozumiem skąd to się bierze.

Modele Lasso i Ridge dają sensowne metryki zarówno na zbiorze treningowym jak i testowym, przy czym Lasso ma podobne wartości na obu zbiorach, a Ridge na istotnie lepsze na zbiorze treningowym niż testowym, co może sugerować lekkie przeuczenie.

Spróbujmy stworzyć model Ridge z większym parametrem regularizacji alpha.

In [ ]:
model_ridge = Ridge(alpha=100)

model_ridge.fit(X_train_processed, y_train)
y_pred_ridge = model_ridge.predict(X_test_processed)
mse_ridge_train = mean_squared_error(y_train, model_ridge.predict(X_train_processed))
r2_ridge_train = r2_score(y_train, model_ridge.predict(X_train_processed))
mse_ridge_test = mean_squared_error(y_test, y_pred_ridge)
r2_ridge_test = r2_score(y_test, y_pred_ridge)

print(f"Ridge Regression - Mean Squared Error, Train: {mse_ridge_train:.2f}, Test: {mse_ridge_test:.2f}")
print(f"Ridge Regression - R^2 Score, Train: {r2_ridge_train:.2f}, Test: {r2_ridge_test:.2f}")

Przy dużej wartości alpha model Ridge osiąga podobne wyniki na zbiorze treningowym i testowym, co sugeruje, że model nie jest już przeuczony.

Zwrómy jeszcze uwagę na współczynniki zmiennych w modelu Lasso, tylko dwie cechy mają niezerowe współczynniki, co oznacza, że model Lasso wybrał tylko te dwie cechy jako istotne dla przewidywania zużycia paliwa.

In [ ]:
lasso_coefs = pd.Series(model_lasso.coef_, index=X_train_processed.columns)
print("Lasso Coefficients:")
print(lasso_coefs.sort_values(ascending=False))